In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from IPython.core.display import display, HTML, clear_output
display(HTML('<style>.container { width:90% !important; }</style>'))
display(HTML('<style>.prompt { min-width:10ex !important; }</style>'))
display(HTML('<style>div#notebook { font-size:12px !important; }</style>'))

pd.set_option('display.max_columns', None)




KeyboardInterrupt



# Выгрузка данных

In [ ]:
april = pd.read_csv('svao_april.csv', sep = ';', low_memory=False, memory_map=True)
march = pd.read_csv('svao_march.csv', sep = ';', low_memory=False, memory_map=True)
febr = pd.read_csv('svao_febr.csv', sep = ';', low_memory=False, memory_map=True)
may = pd.read_csv('svao_may.csv', sep = ';', low_memory=False, memory_map=True)
june = pd.read_csv('svao_june.csv', sep = ';', low_memory=False, memory_map=True)

In [ ]:
data = pd.concat([febr, march,april,may,june])
del febr, march, april, may, june
data.date_ch = pd.to_datetime(data.date_ch)
data.time_ch = pd.to_datetime(data.time_ch)
data['buy_hour'] = data.time_ch.dt.hour
data['CheckLineUID'] = data['CheckLineUID'].astype(str).str.zfill(4)
data['CheckUID'] = data['CheckUID'].astype(str).str.zfill(4)
data.BonusCard_cl = data['BonusCard_cl'].replace('          ', np.nan)
data = data.astype({'BonusCard_cl':'str'})
pd.set_option('display.float_format', lambda x: '%3.f' % x)

# Отберем пользователей, которые посещали ВкусВилл в последний раз 30 и более дней назад, чтобы понять причину их ухода из сети + определить выручку, которую они принесли магазину, чтобы в дальнейшем работать с данной информацией

Выведем крайнюю дату покупки каждым пользователем

In [ ]:
data_vspom = data.groupby(['BonusCard_cl']).agg(
    max_date=('date_ch', 'max')).reset_index()
data_vspom.head()

Выведем максимальную дату, представленню в DF, положим что отсчет идет именно от нее

In [ ]:
data_vspom['max_date_global'] = data['date_ch'].max()

Выведем разницу между крайней датой у исходного DataFrame и выведем только те строки, у которых разница >=30 (месяц). P.S: положим, что месяц - крайняя дата, при котором пользователь уходит из сети

In [ ]:
data_vspom['difference_date'] = data_vspom['max_date_global'] - data_vspom['max_date']
data_vspom = data_vspom[data_vspom['difference_date'] >= '30 days']
data_vspom.shape
#количесвто таких пользователей 180291, что примерно равно половине из тех всех пользователей, у которых есть карта лояльности

В рамках нашего исследования лояльными будем считать тех пользователей, которые ходили во ВкусВилл более 2 раз (2 уникальных чека)

In [ ]:
data_retention = data.groupby(['BonusCard_cl'])['CheckUID'].count().reset_index()
data_vspom = data_vspom.merge(data_retention, on = 'BonusCard_cl')
data_vspom = data_vspom[data_vspom['CheckUID'] >= 2].reset_index(drop = True)
del data_retention

Посмотрим сколько выручки принес пользователь всей сети, чтобы потом определить процент от общей выручки

In [ ]:
data_base_sum = data.groupby('BonusCard_cl')['BaseSum'].sum().reset_index()
data_vspom = data_vspom.merge(data_base_sum, on = 'BonusCard_cl', how = 'inner')
del data_base_sum
294567936.2171673/3155053319.2306204
#Пользователи до оттока из сети ВкусВилл принесли порядка 9% выручки (по всем чекам, что дает нам понять о целесообразности исследования данной выборки)

# Проведем ABC анализ для товаров следующим образом: (За основу берём соотношение 80-15-5)
    

1) Для генеральной совокупности, чтобы определить тренд продуктов в общем

In [ ]:
z1 = data.groupby('id_tov_cl')['BaseSum'].sum().reset_index().sort_values('BaseSum', ascending = False)
z1['total_sum'] = data['BaseSum'].sum()
z1['cum_sum'] = z1['BaseSum'].cumsum()
z1['percent'] = z1['cum_sum'] / z1['total_sum']
z1['group'] = 'C'
z1['group'][z1.percent <= 0.8] = "A"
z1['group'][(z1.percent <= 0.95) & (z1.group != "A")] = "B"

In [ ]:
z1.head()

 2) Для конкретного магазина, чтобы определить расхождения по сравнению с трендом

In [ ]:
z2 = data.groupby(['id_tt_cl','id_tov_cl'])['BaseSum'].sum().reset_index().sort_values(['id_tt_cl','BaseSum'], ascending = [False, False])
z_vspom = z2.groupby('id_tt_cl')['BaseSum'].sum().reset_index()
z2 = z2.groupby(['id_tt_cl', 'id_tov_cl'])['BaseSum'].sum() \
  .groupby(level=0).cumsum().reset_index().sort_values(['id_tt_cl','id_tov_cl','BaseSum'], ascending = [False,False,False])
z2 = z2.merge(z_vspom, on = 'id_tt_cl', how = 'inner')
z2 = z2.rename(columns = ({'BaseSum_x': 'cumsum'}))
z2 = z2.rename(columns = ({'BaseSum_y': 'total_BaseSum'}))
z2['percent'] = z2['cumsum'] / z2['total_BaseSum']
z2['store_group'] = 'C'
z2['store_group'][z2.percent <= 0.8] = "A"
z2['store_group'][(z2.percent <= 0.95) & (z2['store_group'] != "A")] = "B"
del z_vspom

In [ ]:
z2[z2['cumsum'] == z2['total_BaseSum']].shape
#здесь всё правильно посчитано, вывелось 75 магазинов

In [ ]:
z2['store_group'].value_counts()

 3) Для продуктовой корзины каждого человека, чтобы определить предпочтения лояльных пользователей и определить то, что им нравится, определяем по количеству встречаемых продуктов в корзине

In [ ]:
z3 = data.groupby(['BonusCard_cl','id_tov_cl'])['CheckUID'].count().reset_index().sort_values(['BonusCard_cl','CheckUID'], ascending = [False, False]).reset_index(drop = True)
z3 = z3[z3['BonusCard_cl'] != 'nan']
z3 = z3.rename(columns = {'CheckUID':'count_CheckUID'})

In [ ]:
z3.head()

# Объединим полученные результаты с таблицей интересующих нас магазинов

In [ ]:
data_vspom = data_vspom.merge(z3, on ='BonusCard_cl', how = 'inner')
92806901.198194/3155053319.2306204
#эти пользователи генерили нам выручку в 3% по всей сети.
data_vspom = data_vspom[data_vspom['count_CheckUID'] / data_vspom['CheckUID'] >= 0.25]

In [ ]:
data_vspom = data_vspom.merge(z1[['id_tov_cl','group']], on ='id_tov_cl', how = 'inner')

# Перезапишем исходные данные и присоединим информацию по магазинам

In [ ]:
data = data.merge(data_vspom[['difference_date','BonusCard_cl','id_tov_cl','group','CheckUID','count_CheckUID']], on =['BonusCard_cl','id_tov_cl'])
data = data.merge(z2[['id_tt_cl','id_tov_cl','store_group']], on = ['id_tt_cl','id_tov_cl'])

# Посмотрим на пользователей, которые ушли из магазинов из-за смены категории ключевой позиции на полке

In [ ]:
data_top_low = data[(data['group'] != 'C') & (data['store_group'] == 'C')]
#4520 строк с продуктами и товарами, где виден переход из лучшей категории в плохую

Посмотрим на количество категорий, которые были в генеральной совокупности выше категории C

In [ ]:
data_top_low['group'].value_counts()
#как можно заметить, здесь идет отток больше из категорий A

In [ ]:
data_top_low

Посмотрим какое количество продуктов чаще всего встречается в категории A по группе

In [ ]:
data_top_low_id_group_count = data_top_low.groupby(['id_tov_cl', 'id_tt_cl','group'])['CheckLineUID'].count().reset_index().sort_values(['id_tov_cl','CheckLineUID'], ascending = [False,False])

Для простоты рисовки переведем id товара в string

In [ ]:
data_top_low_id_group_count = data_top_low_id_group_count.astype({'id_tov_cl':'str'})
data_top_low_id_group_count = data_top_low_id_group_count.astype({'id_tt_cl':'str'})

In [ ]:
fig = px.scatter(data_top_low_id_group_count,x = 'id_tov_cl', y ='CheckLineUID',color = 'group')
fig.update_layout(
    autosize=False,
    width=2400,
    height=700,)
fig.show()

Явная проблема в товарах категории A, очень много переходов A->C при тренде клиента

Количество проблемных точек в рамках нашего исследования

In [ ]:
data_top_low_id_group_count['id_tov_cl'].nunique()

Сделаем вспомогательную таблицу для определения общей выручки, которую нам могли оставить эти люди

In [ ]:
data = data.astype({'id_tov_cl':'str'})
data = data.astype({'id_tt_cl':'str'})

In [ ]:
data_vspom = data.merge(data_top_low_id_group_count[['id_tov_cl','id_tt_cl','group']], on =['id_tov_cl','id_tt_cl','group'])

In [ ]:
data['BaseSum'].sum()

In [ ]:
32298377.715518482/3155053319.2306204
#искомый 1%, который мог конвертнуться в доп средства для сети

Проверим сколько денег генерли "пострадавшие" люди 

In [ ]:
data_vspom1 = data.merge(data_vspom['BonusCard_cl'], on = 'BonusCard_cl', how ='inner')

In [ ]:
data_vspom1['BaseSum'].sum()